## <a id = '0'> Índice </a>

* [**Entorno**](#1)  
   * [Librerías](#1d1)  
   * [Funciones](#1d2)  
   * [Constantes](#1d3)

* [**Lectura de datos**](#2)


## <a id = '1'> Entorno </a>
[índice](#0)

### <a id = '1d1'> Librerías </a>

In [1]:
import os
import pandas as pd
import numpy as np

from sklearn.preprocessing import  LabelEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
import xgboost as xgb
from xgboost import XGBClassifier

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, ParameterGrid

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, log_loss, classification_report, make_scorer, confusion_matrix
from scipy.stats import ks_2samp

import joblib
# from config import data_folder

In [2]:
os.chdir("../")

In [4]:
MODEL_PATH = "output/models/V3/" 

### <a id = '1d2'> Funciones </a>

In [5]:
from src.utils import table_metrics, params_to_markdown, genera_metricas_markdown

## <a id = '2'> Lectura de datos </a>
[índice](#0)

In [6]:
train_data = pd.read_csv("output/chunk_data/chunk_100/pre_model/train_data.csv")
test_data = pd.read_csv("output/chunk_data/chunk_100/pre_model/test_data.csv")

In [14]:
train_data

,patient_id,label,chunk,I_1,I_2,I_3,I_4,I_5,I_6,I_7,...,V6_91,V6_92,V6_93,V6_94,V6_95,V6_96,V6_97,V6_98,V6_99,V6_100
0,patient_2415,mi,chunk_1,-0.089,-0.059,0.017,-0.035,0.003,0.047,0.004,...,-0.053,-0.037,-0.053,-0.020,0.031,0.022,0.025,0.046,0.031,-0.011
1,patient_2415,mi,chunk_2,-0.007,-0.014,0.024,-0.015,-0.058,0.016,-0.068,...,-0.063,-0.053,-0.026,-0.033,-0.030,-0.025,-0.063,-0.146,-0.120,0.468
2,patient_2415,mi,chunk_3,0.845,0.882,0.277,-0.091,-0.087,-0.090,-0.107,...,-0.176,-0.114,-0.117,-0.101,-0.136,-0.144,-0.146,-0.145,-0.140,-0.131
3,patient_2415,mi,chunk_4,-0.114,-0.112,-0.072,-0.052,-0.111,-0.085,-0.050,...,-0.150,-0.110,-0.096,-0.089,-0.084,-0.065,-0.027,0.006,0.037,0.056
4,patient_2415,mi,chunk_5,0.053,0.054,0.098,0.113,0.133,0.149,0.140,...,0.198,0.195,0.191,0.190,0.161,0.115,0.089,0.067,0.049,0.045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17275,patient_236,other,chunk_6,0.111,0.134,0.171,0.228,0.262,0.292,0.320,...,-0.088,-0.114,-0.135,0.326,1.175,0.997,0.193,-0.001,-0.020,-0.044
17276,patient_236,other,chunk_7,-0.089,-0.073,-0.072,-0.076,-0.046,-0.043,-0.032,...,-0.090,-0.090,-0.090,-0.090,-0.089,-0.081,-0.073,-0.065,-0.058,-0.051
17277,patient_236,other,chunk_8,-0.050,-0.046,-0.029,-0.032,-0.026,-0.062,-0.099,...,-0.095,-0.095,-0.095,-0.095,-0.095,-0.095,-0.099,-0.100,-0.100,-0.105
17278,patient_236,other,chunk_9,-0.056,-0.079,-0.069,-0.056,-0.047,-0.031,-0.037,...,-0.109,-0.111,-0.115,-0.114,-0.121,-0.126,-0.125,-0.124,-0.120,-0.119


In [15]:
#Columnas que no vamos a usar en el modelado
skip_columns = ["patient_id", "label", "chunk"]
le = LabelEncoder()

#Generamos las características y la variable objetivo 
X_train = train_data.drop(columns = skip_columns)
y_train = le.fit_transform(train_data["label"])

X_test = test_data.drop(columns = skip_columns)
y_test = le.fit_transform(test_data["label"])


In [16]:
train_data.shape

(17280, 1203)

In [17]:
recall_macro = make_scorer(recall_score, average='macro') #Todas las clases tienen el mismo peso
f1_score_macro = make_scorer(f1_score, average='macro') #Todas las clases tienen el mismo peso

## Logistic Regression

In [18]:
# Estimador
lr = LogisticRegression(penalty='l2', 
                        C=1e5, 
                        solver='lbfgs', 
                        multi_class='multinomial',
                        random_state = 42)

# Parámetros
params_lr = {
    'penalty': ['l2'],
    'C': [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10]
}
       
# Grid Search
grid_lr = GridSearchCV(estimator=lr,
                       param_grid=params_lr,
                       scoring=recall_macro,
                       cv=5,
                       verbose=1,
                       n_jobs=-1)

In [19]:
grid_lr.fit(X_train, y_train)

Fitting 5 folds for each of 7 candidates, totalling 35 fits


/Users/leongaray/Desktop/MCD_EKG/mcd_venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/Users/leongaray/Desktop/MCD_EKG/mcd_venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/Users/leongaray/Desktop/MCD_EKG/mcd_venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/Users/leongaray/Desktop/MCD_EKG/mcd_venv/lib/python

GridSearchCV(cv=5,
             estimator=LogisticRegression(C=100000.0, multi_class='multinomial',
                                          random_state=42),
             n_jobs=-1,
             param_grid={'C': [1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10],
                         'penalty': ['l2']},
             scoring=make_scorer(recall_score, response_method='predict', average=macro),
             verbose=1)

In [20]:
#Pronóstico
y_pred_grid_lr = grid_lr.best_estimator_.predict(X_test)
y_pred_prob_grid_lr = grid_lr.best_estimator_.predict_proba(X_test)

In [21]:
grid_lr.best_params_

{'C': 1, 'penalty': 'l2'}

In [22]:
print(params_to_markdown(grid_lr.best_params_))

| parámetro   | valor   |
|:------------|:--------|
| C           | 1       |
| penalty     | l2      |


In [23]:
grid_lr.best_score_

0.25833333333333336

In [24]:
grid_lr.best_estimator_

LogisticRegression(C=1, multi_class='multinomial', random_state=42)

In [27]:
joblib.dump(grid_lr, MODEL_PATH + "LR/logistic_regression_model.pkl")

LR_preds = pd.DataFrame({
    "patient_id" : test_data["patient_id"],
    "label" : test_data["label"],
    "y_true" : y_test,
    "pred" : y_pred_grid_lr,
})

LR_probas = pd.DataFrame(y_pred_prob_grid_lr, columns=[f"proba_clase_{c}" for c in grid_lr.classes_])
LR_results = pd.concat([LR_preds, LR_probas], axis=1)
LR_results.to_csv(MODEL_PATH + "LR/predictions.csv", index=False)


In [28]:
table_metrics(y_test,y_pred_grid_lr, y_pred_prob_grid_lr)

,metric,value
0,accuracy,0.251875
1,precision_weighted,0.251922
2,recall_weighted,0.251875
3,f1_weighted,0.251880
4,roc_auc_ovr,0.494501
5,log_loss,1.455907
6,gini_normalized,-0.010997
7,ks_test_clase_0,0.024444
8,ks_test_clase_1,0.035833
9,ks_test_clase_2,0.039722


In [29]:
table_metrics(y_test,y_pred_grid_lr, y_pred_prob_grid_lr)

,metric,value
0,accuracy,0.251875
1,precision_weighted,0.251922
2,recall_weighted,0.251875
3,f1_weighted,0.251880
4,roc_auc_ovr,0.494501
5,log_loss,1.455907
6,gini_normalized,-0.010997
7,ks_test_clase_0,0.024444
8,ks_test_clase_1,0.035833
9,ks_test_clase_2,0.039722


In [30]:
f1_score(y_test, y_pred_grid_lr, average='macro'),

(0.25188000362458157,)

In [31]:
confusion_matrix(y_test, y_pred_grid_lr)

array([[286, 276, 309, 329],
       [289, 317, 278, 316],
       [306, 296, 309, 289],
       [293, 302, 308, 297]])

In [32]:
print(classification_report(y_test, y_pred_grid_lr))

              precision    recall  f1-score   support

           0       0.24      0.24      0.24      1200
           1       0.27      0.26      0.27      1200
           2       0.26      0.26      0.26      1200
           3       0.24      0.25      0.24      1200

    accuracy                           0.25      4800
   macro avg       0.25      0.25      0.25      4800
weighted avg       0.25      0.25      0.25      4800



In [33]:
genera_metricas_markdown(y_test,y_pred_grid_lr, y_pred_prob_grid_lr)

| metric             |      value |
|:-------------------|-----------:|
| accuracy           |  0.251875  |
| precision_weighted |  0.251922  |
| recall_weighted    |  0.251875  |
| f1_weighted        |  0.25188   |
| roc_auc_ovr        |  0.494501  |
| log_loss           |  1.45591   |
| gini_normalized    | -0.0109975 |
| ks_test_clase_0    |  0.0244444 |
| ks_test_clase_1    |  0.0358333 |
| ks_test_clase_2    |  0.0397222 |
| ks_test_clase_3    |  0.0480556 |


|              |   precision |   recall |   f1-score |     support |
|:-------------|------------:|---------:|-----------:|------------:|
| 0            |    0.243612 | 0.238333 |   0.240944 | 1200        |
| 1            |    0.266163 | 0.264167 |   0.265161 | 1200        |
| 2            |    0.256645 | 0.2575   |   0.257072 | 1200        |
| 3            |    0.241267 | 0.2475   |   0.244344 | 1200        |
| accuracy     |    0.251875 | 0.251875 |   0.251875 |    0.251875 |
| macro avg    |    0.251922 | 0.251875 |   0.2

In [ ]:
cm_lr = confusion_matrix(y_test, y_pred_grid_lr)
df_cm_lr = pd.DataFrame(cm_lr,
                         index = [f"Real {label}" for label in grid_lr.classes_],
                         columns = [f"Pred {label}" for label in grid_lr.classes_])
print(df_cm_lr.to_markdown())


## Random Forest Classifier

In [24]:
#Estimador
rfc = RandomForestClassifier(random_state = 42, 
                             n_jobs = -1, 
                             bootstrap = True)

#Parámetros
params_rfc = {'n_estimators': [100, 350, 500],
             'max_features': ['log2', 'sqrt'],
             'max_depth': [5, 10, 20],
             'min_samples_split': [2, 10, 30],
             'min_samples_leaf': [2, 10, 30]}

#Grid Search
grid_rfc = GridSearchCV(estimator=rfc,
                       param_grid=params_rfc,
                       scoring=recall_macro,
                       cv=5,
                       verbose=1,
                       n_jobs=-1)

In [ ]:
grid_rfc.fit(X_train, y_train)

In [ ]:
grid_rfc.best_params_

In [ ]:
print(params_to_markdown(grid_rfc.best_params_))

In [ ]:
grid_rfc.best_score_

In [ ]:
grid_rfc.best_estimator_

In [30]:
#Pronóstico
y_pred_grid_rfc = grid_rfc.predict(X_test)
y_pred_prob_grid_rfc = grid_rfc.predict_proba(X_test)

In [31]:
joblib.dump(grid_rfc, MODEL_PATH + "RFC/random_forest_model.pkl")

RFC_preds = pd.DataFrame({
    "patient" : test_data["patient"],
    "label" : test_data["label"],
    "y_true" : y_test,
    "pred" : y_pred_grid_rfc,
})

RFC_probas = pd.DataFrame(y_pred_prob_grid_rfc, columns=[f"proba_clase_{c}" for c in grid_rfc.classes_])
RFC_results = pd.concat([RFC_preds, RFC_probas], axis=1)
RFC_results.to_csv(MODEL_PATH + "RFC/predictions.csv", index=False)


In [ ]:
table_metrics(y_test,y_pred_grid_rfc, y_pred_prob_grid_rfc)

In [ ]:
confusion_matrix(y_test, y_pred_grid_rfc)

In [ ]:
print(classification_report(y_test, y_pred_grid_rfc))

In [ ]:
genera_metricas_markdown(y_test,y_pred_grid_rfc, y_pred_prob_grid_rfc)

In [ ]:
cm_rfc = confusion_matrix(y_test, y_pred_grid_rfc)
df_cm_rfc = pd.DataFrame(cm_rfc,
                         index = [f"Real {label}" for label in grid_rfc.classes_],
                         columns = [f"Pred {label}" for label in grid_rfc.classes_])
print(df_cm_rfc.to_markdown())


# Gradient Boosting Classifier

In [37]:
#Estimador
gb = GradientBoostingClassifier(learning_rate=0.05, 
                                subsample=0.5, 
                                max_depth=6, 
                                n_estimators=10,
                                 random_state=42,
                                )

#Parámetros
params_gb = {'n_estimators': [1,10,100], 
             'learning_rate' : [0.01,0.05,0.1],
             'subsample' : [0.1,0.5,1.0], 
             'max_depth': [5,10,20],
             'min_samples_split': [2, 10, 30],
             'min_samples_leaf': [2, 10, 30],
             'max_features': ['log2', 'sqrt']}

#Grid Search
grid_gb = GridSearchCV(estimator=gb,
                       param_grid=params_gb,
                       scoring=recall_macro,
                       cv = 5,
                       verbose=1,
                       n_jobs=-1)

In [ ]:
#Entrenamiento
grid_gb.fit(X_train, y_train)

In [ ]:
grid_gb.best_params_

In [ ]:
print(params_to_markdown(grid_gb.best_params_))

In [ ]:
grid_gb.best_score_

In [ ]:
grid_gb.best_estimator_

In [43]:
#Predicciones
y_pred_grid_gb = grid_gb.predict(X_test)
y_pred_prob_grid_gb = grid_gb.predict_proba(X_test)

In [44]:
joblib.dump(grid_rfc, MODEL_PATH + "GB/gradient_boosting_model.pkl")

GB_preds = pd.DataFrame({
    "patient" : test_data["patient"],
    "label" : test_data["label"],
    "y_true" : y_test,
    "pred" : y_pred_grid_gb,
})

GB_probas = pd.DataFrame(y_pred_prob_grid_gb, columns=[f"proba_clase_{c}" for c in grid_gb.classes_])
GB_results = pd.concat([GB_preds, GB_probas], axis=1)
GB_results.to_csv(MODEL_PATH + "GB/predictions.csv", index=False)


In [ ]:
table_metrics(y_test,y_pred_grid_gb, y_pred_prob_grid_gb)

In [ ]:
confusion_matrix(y_test, y_pred_grid_gb)

In [ ]:
print(classification_report(y_test, y_pred_grid_gb))

In [ ]:
genera_metricas_markdown(y_test,y_pred_grid_gb, y_pred_prob_grid_gb)

In [ ]:
cm_gb = confusion_matrix(y_test, y_pred_grid_gb)
df_cm_gb = pd.DataFrame(cm_gb,
                         index = [f"Real {label}" for label in grid_gb.classes_],
                         columns = [f"Pred {label}" for label in grid_gb.classes_])
print(df_cm_gb.to_markdown())

## Naive Bayes

In [50]:
gnb = GaussianNB()

params_gnb = {
    'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
}
#Grid Search
grid_gnb = GridSearchCV(estimator=gnb,
                       param_grid=params_gnb,
                       scoring=recall_macro,
                       cv=5,
                       verbose=1,
                       n_jobs=-1)

In [ ]:
#Entrenamiento
grid_gnb.fit(X_train, y_train)

In [ ]:
grid_gnb.best_params_

In [ ]:
print(params_to_markdown(grid_gnb.best_params_))

In [ ]:
grid_gnb.best_score_

In [55]:
#Predicciones
y_pred_grid_gnb = grid_gnb.predict(X_test)
y_pred_prob_grid_gnb = grid_gnb.predict_proba(X_test)

In [56]:
joblib.dump(grid_gnb, MODEL_PATH + "GNB/Gaussian_Naive_Bayes_model.pkl")

GNB_preds = pd.DataFrame({
    "patient" : test_data["patient"],
    "label" : test_data["label"],
    "y_true" : y_test,
    "pred" : y_pred_grid_gnb,
})

GNB_probas = pd.DataFrame(y_pred_prob_grid_gnb, columns=[f"proba_clase_{c}" for c in grid_gnb.classes_])
GNB_results = pd.concat([GNB_preds, GNB_probas], axis=1)
GNB_results.to_csv(MODEL_PATH + "GNB/predictions.csv", index=False)


In [ ]:
table_metrics(y_test,y_pred_grid_gnb, y_pred_prob_grid_gnb)

In [ ]:
confusion_matrix(y_test, y_pred_grid_gnb)

In [ ]:
print(classification_report(y_test, y_pred_grid_gnb))

In [ ]:
genera_metricas_markdown(y_test,y_pred_grid_gnb, y_pred_prob_grid_gnb)

In [ ]:
cm_gnb = confusion_matrix(y_test, y_pred_grid_gnb)
df_cm_gnb = pd.DataFrame(cm_gnb,
                         index = [f"Real {label}" for label in grid_gnb.classes_],
                         columns = [f"Pred {label}" for label in grid_gnb.classes_])
print(df_cm_gnb.to_markdown())

## XGB Classifier

In [34]:
xgb_base = xgb.XGBClassifier(
    learning_rate=0.1,
    n_estimators=5000,
    eval_metric="merror",
    objective="multi:softprob",
    # early_stopping_rounds=50,
    
)

xgb_base.fit(X_train, y_train,
             eval_set=[(X_test, y_test)],             
             verbose = 1)

# xgb_n_estimator = xgb_base.best_iteration

[0]	validation_0-merror:0.63813
[1]	validation_0-merror:0.62438
[2]	validation_0-merror:0.61104
[3]	validation_0-merror:0.59625
[4]	validation_0-merror:0.58146
[5]	validation_0-merror:0.57542
[6]	validation_0-merror:0.57417
[7]	validation_0-merror:0.57688
[8]	validation_0-merror:0.56854
[9]	validation_0-merror:0.56292
[10]	validation_0-merror:0.55917
[11]	validation_0-merror:0.55583
[12]	validation_0-merror:0.55729
[13]	validation_0-merror:0.55542
[14]	validation_0-merror:0.55083
[15]	validation_0-merror:0.55208
[16]	validation_0-merror:0.54604
[17]	validation_0-merror:0.54875
[18]	validation_0-merror:0.54187
[19]	validation_0-merror:0.54042
[20]	validation_0-merror:0.53625
[21]	validation_0-merror:0.53563
[22]	validation_0-merror:0.53458
[23]	validation_0-merror:0.53583
[24]	validation_0-merror:0.53312
[25]	validation_0-merror:0.53271
[26]	validation_0-merror:0.53271
[27]	validation_0-merror:0.53396
[28]	validation_0-merror:0.53250
[29]	validation_0-merror:0.53354
[30]	validation_0-me

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='merror',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=5000, n_jobs=None,
              num_parallel_tree=None, ...)

In [35]:
#Predicciones
y_pred_grid_xgb_b = xgb_base.predict(X_test)
y_pred_prob_grid_xgb_b = xgb_base.predict_proba(X_test)

In [36]:
table_metrics(y_test,y_pred_grid_xgb_b, y_pred_prob_grid_xgb_b)

,metric,value
0,accuracy,0.523750
1,precision_weighted,0.527639
2,recall_weighted,0.523750
3,f1_weighted,0.525213
4,roc_auc_ovr,0.780134
5,log_loss,1.491370
6,gini_normalized,0.560269
7,ks_test_clase_0,0.284167
8,ks_test_clase_1,0.546944
9,ks_test_clase_2,0.347778


In [37]:
joblib.dump(xgb_base, MODEL_PATH + "XGB/XGB_base_model.pkl")

XGB_base_preds = pd.DataFrame({
    "patient_id" : test_data["patient_id"],
    "label" : test_data["label"],
    "y_true" : y_test,
    "pred" : y_pred_grid_xgb_b,
})

XGB_base_probas = pd.DataFrame(y_pred_prob_grid_xgb_b, columns=[f"proba_clase_{c}" for c in xgb_base.classes_])
XGB_base_results = pd.concat([XGB_base_preds, XGB_base_probas], axis=1)
XGB_base_results.to_csv(MODEL_PATH + "XGB/predictions_base.csv", index=False)


In [38]:
genera_metricas_markdown(y_test,y_pred_grid_xgb_b, y_pred_prob_grid_xgb_b)

| metric             |    value |
|:-------------------|---------:|
| accuracy           | 0.52375  |
| precision_weighted | 0.527639 |
| recall_weighted    | 0.52375  |
| f1_weighted        | 0.525213 |
| roc_auc_ovr        | 0.780134 |
| log_loss           | 1.49137  |
| gini_normalized    | 0.560269 |
| ks_test_clase_0    | 0.284167 |
| ks_test_clase_1    | 0.546944 |
| ks_test_clase_2    | 0.347778 |
| ks_test_clase_3    | 0.488056 |


|              |   precision |   recall |   f1-score |    support |
|:-------------|------------:|---------:|-----------:|-----------:|
| 0            |    0.412201 | 0.416667 |   0.414422 | 1200       |
| 1            |    0.68011  | 0.618333 |   0.647752 | 1200       |
| 2            |    0.457929 | 0.471667 |   0.464696 | 1200       |
| 3            |    0.560317 | 0.588333 |   0.573984 | 1200       |
| accuracy     |    0.52375  | 0.52375  |   0.52375  |    0.52375 |
| macro avg    |    0.527639 | 0.52375  |   0.525213 | 4800       |
| weighted a

In [39]:
cm_xgb_base = confusion_matrix(y_test, y_pred_grid_xgb_b)
df_cm_xgb_base = pd.DataFrame(cm_xgb_base,
                         index = [f"Real {label}" for label in xgb_base.classes_],
                         columns = [f"Pred {label}" for label in xgb_base.classes_])
print(df_cm_xgb_base.to_markdown())

|        |   Pred 0 |   Pred 1 |   Pred 2 |   Pred 3 |
|:-------|---------:|---------:|---------:|---------:|
| Real 0 |      500 |      228 |      250 |      222 |
| Real 1 |      233 |      742 |      174 |       51 |
| Real 2 |      250 |      103 |      566 |      281 |
| Real 3 |      230 |       18 |      246 |      706 |


In [40]:
print(params_to_markdown(xgb_base.get_params()))

| parámetro          | valor          |
|:-------------------|:---------------|
| objective          | multi:softprob |
| enable_categorical | False          |
| eval_metric        | merror         |
| learning_rate      | 0.1            |
| missing            | nan            |
| n_estimators       | 5000           |
